In [38]:
from PyPDF2 import PdfReader
from tqdm import tqdm
import re, os, time, glob # Para buscar archivos en un directorio, mejor que os.listdir

class BOCPdftoTxt:
    """
    Clase para vectorizar archivos PDF del Banco de Chile (BOC).

    Args:
        carpeta_pdf (str, optional): Ruta de la carpeta que contiene los archivos PDF. 
            Por defecto, se utiliza 'default_boc'.
        carpeta_txt (str, optional): Ruta de la carpeta donde se guardarán los archivos de texto. 
            Por defecto, se utiliza 'default_boc_txt'.

    Attributes:
        _path_txt (str): Ruta de la carpeta donde se guardarán los archivos de texto.
        _path_pdf (str): Ruta de la carpeta que contiene los archivos PDF.
        _tiempo_total (float): Tiempo total de la conversión en segundos.
    """
    
    def __init__(self, carpeta_pdf='default_boc', carpeta_txt='default_boc_txt'):
        self._path_txt = carpeta_txt
        self._path_pdf = carpeta_pdf
        self._tiempo_total = 0

    def _convertir_pdf_a_texto(self, pdf_file):
        """
        Convierte un archivo PDF a texto.

        Args:
            pdf_file (str): Ruta del archivo PDF.

        Returns:
            str: El texto extraído del archivo PDF.
        """
        doc_pdf = open(pdf_file, 'rb') # Abrir el archivo en modo lectura binaria
        pdf_reader = PdfReader(doc_pdf) # Crear un objeto PdfReader

        doc_text = ""
        for page in pdf_reader.pages:
            doc_text += page.extract_text()
            
        # Unir palabras divididas por un guión al final de una línea
        doc_text = re.sub('-\n', '', doc_text)
            
        return doc_text

    def _guardar_texto_en_txt(self, texto, pdf_file):
        """
        Guarda el texto en un archivo de texto.

        Args:
            texto (str): El texto a guardar.
            pdf_file (str): Ruta del archivo PDF.

        """
        if not os.path.exists(self._path_txt):
            os.makedirs(self._path_txt)
            
        base_name = os.path.basename(pdf_file) # Obtén el nombre base del archivo
        txt_name = os.path.splitext(base_name)[0] + '.txt' # Cambia la extensión a .txt

        with open(f'{self._path_txt}/{txt_name}', "w") as file:
            file.write(texto)
            
    def run(self):
        """
        Ejecuta la conversión de todos los archivos PDF a texto.
        """
        self._tiempo_total = 0
        
        tiempo_inicio = time.time() # Iniciar el cronómetro
        
        pdf_files = glob.glob(self._path_pdf  + "/*.pdf") # Busca todos los archivos .pdf en la carpeta

        for pdf_file in tqdm(pdf_files): # Itera sobre cada archivo .pdf
            text = self._convertir_pdf_a_texto(pdf_file)
            self._guardar_texto_en_txt(text, pdf_file)
        
        tiempo_final = time.time() # Detener el cronómetro

        self._tiempo_total = (tiempo_final - tiempo_inicio) # Calcular el tiempo total
    
    def tiempo(self):
        """
        Obtiene el tiempo total de la conversión.

        Returns:
            float: El tiempo total de la conversión en minutos.
        """
        if self._tiempo_total == 0:
            return "No se ha convertido ningún documento.", None
        elif self._tiempo_total < 60: #segundos
            return round(self._tiempo_total,2), 'seg'
        elif self._tiempo_total < 3600: #minutos
            return round(self._tiempo_total/60,2), 'min'
        elif self._tiempo_total < 86400: #horas
            return round(self._tiempo_total/3600,2), 'horas'
        elif self._tiempo_total < 2592000: #días
            return round(self._tiempo_total/86400,2), 'días'

        

In [39]:
prueba = BOCPdftoTxt(carpeta_pdf='../boc', carpeta_txt='../boc_txt')
prueba.run()
prueba.tiempo()

100%|██████████| 999/999 [00:18<00:00, 52.63it/s]


(18.99, 'seg')